In [ ]:
import numpy as np
import yfinance as yf
from scipy.optimize import minimize


In [ ]:
q = 0.99
prct = 95

In [ ]:
data = yf.download("TSLA", start="2022-01-05", end="2026-01-01")["Close"]

In [ ]:
returns = data.pct_change().dropna().values
L = np.abs(returns[returns<0])
u = np.percentile(L, prct)
P_u = len(L[L>u])/len(L)
print(u, P_u)

Y = L[L > u] - u
print(Y)

In [ ]:
from scipy.stats import genpareto

In [ ]:
def gpd_neg_log_likelihood(params, Y):
  xi, beta = params
  if beta <= 0:
    return np.inf
  if abs(xi) < 1e-6:
    nll = np.sum(np.log(beta) + Y/beta)

  else:
    t = 1 + xi * Y / beta
    if np.any(t <= 0):
      return np.inf
    nll = np.sum(np.log(beta) + (1/xi+ 1) * np.log(t))

  return nll

In [ ]:
init_params = [0.1, np.mean(Y)]
use_minimize = False
if use_minimize:
  results = minimize(
      gpd_neg_log_likelihood,
      init_params,
      args=(Y,),
      method="L-BFGS-B",
      bounds=[(-1, 2), (1e-6, None)]
  )
  xi, beta = results.x
else: 
  xi, loc, beta = genpareto.fit(Y, floc=0)
print(xi, beta)

In [ ]:
VaR_q = u + beta/xi * (((1-q)/P_u)**-xi - 1)
CTE_q = VaR_q / (1 - xi) + (beta-xi*u)/(1-xi)


In [ ]:
import math
def get_tail_moments_gpd(beta, xi, order):
  moments = []

  for k in range(1, order + 1):
    if xi >= 1 / k:
      moments.append(np.inf)
      continue

    moment = np.exp(
      math.lgamma(k + 1) + k * np.log(beta) - sum(np.log(1 - j * xi) for j in range(1, k + 1))
    )
    moments.append(round(moment, 8))

  return moments

In [ ]:
order = 4
tcm = get_tail_moments_gpd(beta, xi, order)

In [ ]:
print(f"Threshold u: {u:.4%}, Exceedance fraction P_u: {P_u:.4f}")
print(f"Estimated xi: {xi:.4f}, beta: {beta:.4f}")
print(f"EVT VaR 99%: {VaR_q:.2%}, EVT CTE 99%: {CTE_q:.2%}")
print(f"TCM up to {order} order: {tcm}")